### Import the necessary database

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
#In[2]:
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprosess

In [ ]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
import os

dir_ICV_segment = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure3/data/segment_trends/'

vars_name = np.arange(10, 74, 1).astype(str)

# Create an empty xarray.Dataset
combined_ds = xr.Dataset()

# Loop over each segment length and load the data directly into the dataset
for segment_length in vars_name:
    file_path = dir_ICV_segment + f'GSAT_HadCRUT5_Internal_Variability_trend_{segment_length}yr_segments_1850_2022.nc'
    
    # Load each segment dataset
    ds = xr.open_mfdataset(file_path, chunks={'segment': 1})
    
    # Add the variable directly to the combined dataset with a meaningful name
    var_name = f'ICV_segments_{segment_length}yr_trend'
    combined_ds[var_name] = ds[f'ICV_segments_{segment_length}yr_trend']

# Now, `combined_ds` holds all the data
print(combined_ds)

In [ ]:
combined_ds

In [ ]:
# print(np.arange(10,31,1).astype(str))

In [ ]:
# import os

# dir_ICV_segment = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure3/data/segment_trends/'


# ICV_data = {}
# for vars in vars_name:
#     file_path = dir_ICV_segment + f'GSAT_HadCRUT5_Internal_Variability_trend_{vars}yr_segments_1850_2022.nc'
#     print(f'Checking file: {file_path}')
    
#     # Check if the file exists
#     if os.path.exists(file_path):
#         ICV_data[vars] = xr.open_mfdataset(file_path, chunks={'segment': 1})
#     else:
#         print(f"File not found: {file_path}")


In [ ]:
# merged_ds = xr.merge([ds for ds in ICV_data.values()])

In [ ]:
# merged_ds

In [ ]:
# define function to calculate the standard deviation of the trend pattern of each interval of segments
def std_trend_pattern(data):
    """
    data: 4D array with dimensions [year, lat, lon, segment]
    segment_length: length of each segment in years
    """
    # calculate the standard deviation of the trend pattern of each interval of segments
    std_trend_pattern = np.nanstd(data, axis=0)
    
    return std_trend_pattern

In [ ]:
 # Calculate the standard deviation of the trend pattern of each interval of segments
for segment_length in vars_name:
    var_name = f'ICV_segments_{segment_length}yr_trend'

    # Apply the custom function using apply_ufunc along the 'segment' dimension
    combined_ds[f'{var_name}_std'] = xr.apply_ufunc(
        std_trend_pattern,                       # Custom function
        combined_ds[var_name],                   # Input data
        input_core_dims=[['segment']],           # Dimension to apply the function along
        output_core_dims=[[]],                   # Output does not keep 'segment' dimension
        vectorize=True,                          # Apply element-wise
        dask='parallelized',                     # Enable parallel computation with Dask
        output_dtypes=[float],                   # Specify output data type
        dask_gufunc_kwargs={'allow_rechunk': True}  # Allow rechunking if necessary
    )


In [ ]:
combined_ds

In [ ]:
# save the std of trend pattern of each trend years
dir_output = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure3/data/ICV_STD_whole/'

for segment_length in vars_name:
    var_name = f'ICV_segments_{segment_length}yr_trend'
    combined_ds[f'{var_name}_std'].to_netcdf(dir_output + f'GSAT_HadCRUT5_Internal_Variability_trend_{segment_length}yr_segments_1850_2022_std.nc')


### Plot the trend of the ICV patterns

In [ ]:
plt.rcParams['figure.figsize'] = (8, 10)
plt.rcParams['font.size'] = 16
# plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.major.right'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['savefig.transparent'] = True

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

In [ ]:
def plot_trend(trend_data, lats, lons, levels=None, extend=None, cmap=None, 
                                 title="", ax=None, show_xticks=False, show_yticks=False):
    """
    Plot the trend spatial pattern using Robinson projection with significance overlaid.

    Parameters:
    - trend_data: 2D numpy array with the trend values.
    - lats, lons: 1D arrays of latitudes and longitudes.
    - p_values: 2D array with p-values for each grid point.
    - GMST_p_values: 2D array with GMST p-values for each grid point.
    - title: Title for the plot.
    - ax: Existing axis to plot on. If None, a new axis will be created.
    - show_xticks, show_yticks: Boolean flags to show x and y axis ticks.
    
    Returns:
    - contour_obj: The contour object from the plot.
    """
# Create a new figure/axis if none is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(20, 15), subplot_kw={'projection': ccrs.Robinson()})
        ax.set_global()
        
    contour_obj = ax.contourf(lons, lats, trend_data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree(central_longitude=0))
    # Plot significance masks with different hatches
    # ax.contourf(lons, lats, significance_mask, levels=[0.05, 1.0],hatches=['///'], colors='none', transform=ccrs.PlateCarree())

    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, linewidth=1, color='gray', alpha=0.35)

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 16}
    gl.ylabel_style = {'size': 16}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    
    ax.set_title(title, loc='center', fontsize=24, pad=5.0)

    return contour_obj

In [ ]:
# define an asymmetric colormap
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colors import BoundaryNorm

intervals = [0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]

# Normalizing the intervals to [0, 1]
min_interval = min(intervals)
max_interval = max(intervals)
normalized_intervals = [(val - min_interval) / (max_interval - min_interval) for val in intervals]

# colors = ['#2616D3', '#005EFF', '#0084FF', '#00A2FF', '#00BCDB', (1.0, 1.0, 1.0, 1.0),(1.0, 1.0, 1.0, 1.0),(1.0, 0.8, 0.5, 1.0),
#     (1.0, 0.803921568627451, 0.607843137254902, 1.0), (1.0, 0.6000000000000001, 0.20000000000000018, 1.0),(1.0, 0.4039215686274509, 0.0, 1.0),(0.8999999999999999, 0.19999999999999996, 0.0, 1.0),
#     (0.7470588235294118, 0.0, 0.0, 1.0), (0.6000000000000001, 0.0, 0.0, 1.0),(0.44705882352941173, 0.0, 0.0, 1.0),(0.30000000000000004, 0.0, 0.0, 1.0),(0.14705882352941177, 0.0, 0.0, 1.0),
#     (0.0, 0.0, 0.0, 1.0)]

# Creating a list of tuples with normalized positions and corresponding colors
# color_list = list(zip(normalized_intervals, colors))

# # Create the colormap
# custom_cmap = LinearSegmentedColormap.from_list('my_custom_cmap', color_list)

# # Create a normalization
# norm = Normalize(vmin=min_interval, vmax=max_interval)

In [ ]:
import seaborn as sns
import palettable
from palettable.colorbrewer.diverging import RdBu_11_r
import matplotlib.colors as mcolors

cmap = mcolors.ListedColormap(palettable.cmocean.sequential.Amp_20.mpl_colors)

In [ ]:
lat = combined_ds['lat'].values
lon = combined_ds['lon'].values

titles = ["55yr",  "60yr", "30yr"]

import cartopy.util as cutil
# levels = np.arange(-0.2, 0.25, 0.025)
# Define the GridSpec
fig,ax = plt.subplots(1, 3, figsize=(20, 12), subplot_kw={'projection': ccrs.Robinson(180)})

levels = np.arange(0.0, 1.1, 0.1)

trend_data_10yr = combined_ds['ICV_segments_55yr_trend_std'].values
trend_with_cyclic_10yr, lon_with_cyclic = cutil.add_cyclic_point(trend_data_10yr, coord=lon)
contour_obj = plot_trend(trend_with_cyclic_10yr, lat, lon_with_cyclic,levels=levels,extend='max',
                    cmap=cmap,
                    title=titles[0], ax=ax[0], show_xticks = True, show_yticks = True)

trend_data_30yr = combined_ds['ICV_segments_60yr_trend_std'].values
trend_with_cyclic_30yr, lon_with_cyclic = cutil.add_cyclic_point(trend_data_30yr, coord=lon)
contour_obj1 = plot_trend(trend_with_cyclic_30yr, lat, lon_with_cyclic,levels=levels,extend='max',
                    cmap=cmap,
                    title=titles[1], ax=ax[1], show_xticks = True, show_yticks = False)

# trend_data_60yr = combined_ds['ICV_segments_30yr_trend_std'].values
# trend_with_cyclic_60yr, lon_with_cyclic = cutil.add_cyclic_point(trend_data_60yr, coord=lon)
# contour_obj2 = plot_trend(trend_with_cyclic_60yr, lat, lon_with_cyclic,levels=levels,extend='max',
#                     cmap=cmap,
#                     title=titles[2], ax=ax[2], show_xticks = True, show_yticks = False)
# sns.color_palette("rocket_r", as_cmap=True)
# Add horizontal colorbars
cbar_ax = fig.add_axes([0.3, 0.3, 0.5, 0.025])
cbar = plt.colorbar(contour_obj, cax=cbar_ax, orientation='horizontal', extend='max')
cbar.ax.tick_params(labelsize=18)
cbar.set_label('°C/decade', fontsize=22)

plt.tight_layout()
fig.savefig('ICV_std_patterns(55-60yrs).png', dpi=300, bbox_inches='tight')
plt.show()